In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Khởi tạo SparkSession với hỗ trợ Delta Lake
builder = SparkSession.builder \
    .appName("DeltaHistoryViewer") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


25/05/24 15:05:17 WARN Utils: Your hostname, LAPTOP-PMSG8QVF resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/24 15:05:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfd8fa66-08fe-4879-827b-50f5bebe818f;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 135ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0 

In [6]:
from delta.tables import DeltaTable

# Danh sách thư mục chứa Delta Tables
delta_paths = [
    "hdfs://localhost:9000/user/minhhieu/delta/bronze/from_api",
    "hdfs://localhost:9000/user/minhhieu/delta/bronze/from_csv",
    "hdfs://localhost:9000/user/minhhieu/delta/silver/merged_weather",
    "hdfs://localhost:9000/user/minhhieu/delta/gold/weather_features"
]

# Duyệt qua từng bảng và in lịch sử Delta log
for path in delta_paths:
    try:
        print(f"\n📁 Lịch sử của Delta Table: {path}")
        delta_table = DeltaTable.forPath(spark, path)
        history_df = delta_table.history()  # Mặc định là 30 phiên bản gần nhất
        history_df.select("version", "timestamp", "operation", "operationMetrics", "userMetadata").show(truncate=False)
    except Exception as e:
        print(f"⚠️ Không thể đọc lịch sử Delta Table tại: {path}")
        print("Lý do:", str(e))



📁 Lịch sử của Delta Table: hdfs://localhost:9000/user/minhhieu/delta/bronze/from_api
+-------+-----------------------+----------------+--------------------------------------------------------------------------------------+------------+
|version|timestamp              |operation       |operationMetrics                                                                      |userMetadata|
+-------+-----------------------+----------------+--------------------------------------------------------------------------------------+------------+
|526    |2025-05-24 14:36:04.401|STREAMING UPDATE|{numRemovedFiles -> 0, numOutputRows -> 2, numOutputBytes -> 4271, numAddedFiles -> 1}|NULL        |
|525    |2025-05-24 14:36:03.964|STREAMING UPDATE|{numRemovedFiles -> 0, numOutputRows -> 2, numOutputBytes -> 4271, numAddedFiles -> 1}|NULL        |
|524    |2025-05-24 14:36:03.512|STREAMING UPDATE|{numRemovedFiles -> 0, numOutputRows -> 2, numOutputBytes -> 4271, numAddedFiles -> 1}|NULL        |
|523    

In [5]:
delta_path = "hdfs://localhost:9000/user/minhhieu/delta/gold/weather_features"
df_old = spark.read.format("delta").option("versionAsOf", 0).load(delta_path)
df_old.write.format("delta").mode("overwrite").save("hdfs://localhost:9000/user/minhhieu/delta/gold/weather_features_v0")


25/05/24 15:09:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [6]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

# Khởi tạo SparkSession
builder = SparkSession.builder \
    .appName("DescribeDeltaHistory") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Đường dẫn Delta Table
delta_path = "hdfs://localhost:9000/user/minhhieu/delta/silver/merged_weather_02"

# Load bảng Delta
delta_table = DeltaTable.forPath(spark, delta_path)

# Hiển thị lịch sử ghi: operation = WRITE hoặc STREAMING UPDATE
history_df = delta_table.history()  # mặc định trả về 30 version gần nhất

# In chi tiết version, thời gian, loại ghi
history_df.select("version", "timestamp", "operation", "operationMetrics").show(truncate=False)


+-------+-----------------------+----------------+--------------------------------------------------------------------------------------+
|version|timestamp              |operation       |operationMetrics                                                                      |
+-------+-----------------------+----------------+--------------------------------------------------------------------------------------+
|530    |2025-05-25 02:30:21.857|WRITE           |{numFiles -> 2, numOutputRows -> 1, numOutputBytes -> 5265}                           |
|529    |2025-05-25 02:30:21.201|WRITE           |{numFiles -> 2, numOutputRows -> 1, numOutputBytes -> 5265}                           |
|528    |2025-05-25 02:30:20.312|WRITE           |{numFiles -> 2, numOutputRows -> 1, numOutputBytes -> 5265}                           |
|527    |2025-05-25 02:30:16.375|WRITE           |{numFiles -> 2, numOutputRows -> 1, numOutputBytes -> 5265}                           |
|526    |2025-05-25 02:24:26.78 |S

In [7]:
from pyspark.sql.functions import col

# Đếm số lần mỗi loại thao tác xuất hiện
history_df.groupBy("operation").count().show()


+----------------+-----+
|       operation|count|
+----------------+-----+
|           WRITE|    4|
|STREAMING UPDATE|  527|
+----------------+-----+

